# Quantifying Total Error Around A Model (COMPLETE)

## Chapter 6.1-6.3 Overview Notebook

<div class="teacher-note">

**Teacher Note:**
Before distributing this notebook to students, be sure to fill in a value for class_id in the cell below. We suggest using something that will be unique to you, e.g. you email address. (Just replace where we have entered `teacher@myschool.com.`)
    
This will be the way we pull out the data just for your students. 
    
</div>

In [ ]:
# teacher enter a class_id here
class_id <- "teacher@myschool.com" # put any unique id here (e.g., teacher email address)

# run this to set up the notebook

# run this code to load the R packages we will use
#remotes::install_version("coursekata", "0.19.0", upgrade="never")
library(coursekata)
library(gridExtra)

# set styles
css <- suppressWarnings(readLines("https://raw.githubusercontent.com/jimstigler/jupyter/master/ck_jupyter_styles_v2.css"))
IRdisplay::display_html(sprintf('<style>%s</style>', paste(css, collapse = "\n")))

# create two data frames: student_1 and student_2
time_sec <- c(7.4, 6, 7.5, 6.4, 6.7)
trial <- c(1:5)
student1 <- data.frame(trial, time_sec)

time_sec <- c(7.4, 6.7, 7.1, 6.2, 7.2, 6.4, 6.5, 7.2, 6.6, 7.4)
trial <- c(1:10)
student2 <- data.frame(trial, time_sec)

# set some variables for this notebook
condition <- "guess" # set as "timer" or "guess"
user_id <- Sys.getenv("JUPYTERHUB_USER") # this gets the users Jupyter ID

# pull in gf_resid and gf_squaresid for now
source("https://raw.githubusercontent.com/UCLATALL/stopwatch/refs/heads/main/gf_resid.R")
source("https://raw.githubusercontent.com/UCLATALL/stopwatch/refs/heads/main/gf_square_resid.R")


<div class="teacher-note">
<b>Chapter Overview:</b> In the previous chapter we defined error as the residuals from the model’s prediction for an individual data point to the actual value in the data. In this chapter students will develop tools and concepts for quantifying total error around a model, aggregating across all data points. They also will be introduced to ways of using the distribution of errors for placing individual scores in context and for making different kinds of predictions. Finally, they will explore the idea of modeling a distribution with the normal distribution.
    
<b>Section Goals:</b> In this section students are introduced to the important concepts of Sum of Squares and variance as ways of quantifying how much total error there is around a model’s predictions. 

- Students learn that because the sum of the residuals around the mean will always be 0, it is not useful as a way of calculating total error. We typically use the sum of squared residuals because it is minimized at the mean of the distribution of any quantitative variable.
- Students learn that squared residuals can be represented as squares on the coordinate plane, and that the sum of squares is literally the sum of the areas of the squares formed by squaring residuals.
- Students also are introduced to the concept of variance, which is based on the sum of squares, but adjusted by sample size to be per observation, making it comparable across different samples. 
- Students learn to use R to calculate sum of squares and variance from a data frame directly and by using the supernova() function.
    
</div>

## 1 Why Do We Build Models? To Reduce Error

When we build models in statistics, one of our big goals is to reduce error. No model is ever going to be perfect. It's predictions will always be a little off from the data. But some models do a better job than others.

So far we have learned to make a very simple model: the empty model (on the left). Later we will learn to make more complex models (like the one on the right). Both of these models are based on the same data points. Just looking at them, you may think that one model has more error than the other. But how much more?

In order to compare the two models accurately, we need a **measure** of how much total error there is around a model. Developing such a measure is the focus of this lesson, starting with the empty model.

<img src="https://coursekata-course-assets.s3.us-west-1.amazonaws.com/UCLATALL/czi-stats-course/6.1-6.3-overview-empty-complex.jpg" width="90%">


<div class="discussion-question">
    
### 1.1 Discussion Question: Which model has less error and how can you tell? Why can't we just use the sum of the residuals as a measure of total error? 
    
</div>

<div class="teacher-note">

**Sample Responses:**  
- The complex model seems like it has less error because the residuals are shorter.
- The positive and negative residuals cancel out. 
- The total will always be zero. And it's not that the empty model literally has 0 error.

If students think summing residuals might work, you might consider asking:   
- The sum of residuals from the empty model was 0. Does that mean the model makes no error? 
- If we compared two different models and both had sums of zero, how would we decide which is better?

The goal is to help them see that “balancing” is not the same as “small error.” Hopefully students see a need for a measure of error that doesn’t cancel positives and negatives. 

</div>

## 2 Estimating Elapsed Time

In order to start developing a measure of error, we need some data we can model.  

### 2.1 Let’s collect our own!

In the cell below we have a stopwatch. But this is not a normal stopwatch. On this stopwatch, after you click the **Start** button you won't be able to read the elapsed time until your click on **Stop**. Let's see how it works. Try using the stopwatch to guess when exactly 7 seconds are up. Try it a few times. 

<div class="teacher-note">

**Teacher Note:**

Project the timer on a screen. Let a few students come up one at a time and try to use the stopwatch to guess when exactly 7 seconds are up. Alternatively, if students are working in their own notebooks, they can all try the stopwatch game.
    
You can ask students about their strategies for estimating elapsed time (their internal "data generating process"). Many will count—sometimes using "1-mississippi, 2-mississippi." 

</div>

In [ ]:
# run this cell
IRdisplay::display_html(sprintf(
  '<iframe src="https://uclatall.github.io/stopwatch/stopwatch-v3.html?save_data=false&progress=hide" width="500" height="210" sandbox="allow-scripts allow-same-origin"></iframe>'
))

<div class="discussion-question">
    
### 2.2 Discussion Questions: Let's say someone tried the guessing game five times. Could you use their data to make a model to predict what their guess would be for a sixth time? 

- How would you make the model?
- What does it mean to "fit" a model? 
- What does it mean for a model to be the "best fit" for the data?
    
</div>

<div class="teacher-note">

**Teacher Note:**
- Some students might say "use the empty model" or "average" or "mean" mostly because that is all they know so far.
- Remind students that when we say a model “fits the data,” we’re looking at how far off each prediction is from the actual guess (the residuals). The mean would be the best-fitting one parameter model because it perfectly balances the residuals. 
    
</div>

### 2.3 Let's collect 5 trials of data from a single student.

We need a volunteer to do 5 trials guessing an elapsed time of 7 seconds. The stopwatch below will collect the data from each trial and save it in a Google sheet.  

**Important: This time we've made the task a little harder.** You won’t get to see how well the volunteer did until all 5 trials are complete and we pull the data into R. This way we’ll have a small dataset we can use to fit the empty model and explore total error.

In [ ]:
# then run this code to render the stopwatch
IRdisplay::display_html(sprintf(
  '<iframe src="https://uclatall.github.io/stopwatch/stopwatch-v3.html?condition=%s&class_id=%s&user_id=%s&trials=5&progress=hide&result=hide" width="500" height="210" sandbox="allow-scripts allow-same-origin"></iframe>',
  condition, class_id, user_id
))

### 2.4 How well did they do? Run the code below to import the data into R

The code below imports data for the five trials into an R data frame called `volunteer_student`

In [ ]:
# run this code to import the most recent session from your class
sheet_url <- "https://docs.google.com/spreadsheets/d/e/2PACX-1vQRi4VBEFYGz_HnJXBsxlawbqM4vg5vgullb-iIqc5-dOpqEBCck12IS7lPD7bEdoxe8VPKYKe86Db4/pub?output=csv"
volunteer_student <- utils::read.csv(sheet_url, header = TRUE, stringsAsFactors = FALSE) %>%
  mutate(date = as.POSIXct(date, format = "%Y-%m-%dT%H:%M:%OSZ", tz = "UTC")) %>%
  filter(user_id == !!user_id) %>%
  filter(session_id == session_id[which.max(date)]) %>%
  select(user_id, trial, time_sec, date)

# prints out the recent_data
volunteer_student

<div class="guided-notes">
    
### 2.5 What does each row of the data frame represent? How about each column? (Put an asterisk next to the outcome variable.)
    
</div>

<div class="teacher-note">

**Teacher Note:**
    
Note that `trial` shows the trial number of the 5 trials. You should see 5 rows of data.
    
</div>

## 3 Fit the Empty Model of `time_sec`

### 3.1 Let's look at data from someone called `student1` and then create a scatter plot.

Let’s look at some data collected from a previous student who tried this task. Run the code below to take a look at the `student1` data frame. 

Then make a scatter plot with `time_sec` on the y-axis (the outcome variable) and `trial` on the x-axis. How well did this person do?


In [ ]:
# run this cell
str(student1)

# graph time_sec by trial

# COMPLETE
gf_point(time_sec ~ trial, data = student1) 

<div class="guided-notes">
    
### 3.2 Write code to fit the empty model of `time_sec` and save it as `empty_model1`, and then print out the parameter estimate ($b_0$)
    
</div>

In [ ]:
# code here

# COMPLETE
empty_model1 <- lm(time_sec ~ NULL, data = student1)
empty_model1

<div class="guided-notes">
    
### 3.3 Write the best-fitting model in GLM notation. 
</div>

<div class="teacher-note">

**Sample response:** 
    
$Y_i=6.8+e_i$
</div>

<div class="guided-notes">
    
### 3.4 Draw the best-fitting empty model onto the scatter plot provided
    
</div>


In [ ]:
# to see what the drawing should look like, run this 
gf_point(time_sec ~ trial, data = student1) %>%
  gf_model(empty_model1) 

<div class="discussion-question">
    
### 3.5 Discussion Question: How would we use this model to make a prediction about the next trial?
    
</div>

<div class="teacher-note">

**Sample Response:** 

- We can use this number (6.8) as our prediction. On their next trial, we predict this person would stop the timer at 6.8 seconds.
    
</div>

## 4 Measuring Total Error

We have fit the empty model for `student1`. We know there is error around the model prediction, evidenced by the fact that none of the estimates are exactly 6.8 seconds. But how much *total error*? How can we measure total error?  

We already know one method that won’t work: just adding up the residuals. That always comes out to zero for the empty model.


<div class="discussion-question">
    
### 4.1 Discussion Question: What are some other ideas for how to calculate the total error around a model?
    
</div>

In [ ]:
# run this
gf_point(time_sec ~ trial, data = student1) %>%
  gf_model(empty_model1) %>%
  gf_resid(empty_model1, color="red")

### 4.2 One idea: Sum the absolute values of the residuals

A good idea is to add up the absolute values of the residuals: `sum(abs(resid(empty_model1)))` In effect, this sums up the distances of each data point from the model prediction.

- ✅ Absolute value solves the canceling problem (positive and negative errors both become positive).
- ❌ The problem with this solution is that the sum of absolute residuals is not minimized at the mean. The mean would not be the *best-fitting* empty model if this were our measure of error.

In [ ]:
# If you want to explore sum of absolute residuals, 
# try uncommenting each of these lines of code one at a time

# resid(empty_model1)
# abs(resid(empty_model1))
# sum(abs(resid(empty_model1)))

### 4.3 The most common solution: Sum of Squares Total (SST)

Instead of adding residuals or absolute residuals, the most common measure of total error is the sum of the squared residuals. This is calculated by squaring each residual, then adding up the squares.

- ✅ Squaring solves the canceling problem (positive and negative errors both become positive).
- ✅ Squaring also ensures the mean is the best-fitting empty model.

This measure is called the Sum of Squares (SS). When we’re talking about the total error around the empty model, we call it the Sum of Squares Total (SST).

<div class="guided-notes">
    
### 4.4 We've learned how to represent residuals on a graph. How would we represent squared residuals on a graph? Draw how you think you could do that.

</div>


In [ ]:
# run this after students predict
gf_point(time_sec ~ trial, data = student1) %>%
  gf_model(empty_model1) %>%
  gf_resid(empty_model1) %>%
  gf_squaresid(empty_model1, color = "blue")

<div class="discussion-question">
    
### 4.5 Discussion Question: In sum of squares, what exactly is being summed?
    
As a contrast, also consider, in sum of residuals, what exactly is being summed?
    
</div>

<div class="teacher-note">

**Sample Responses:**  
- In the sum of squares, we are adding up the **areas** of squares, each side equal to the length of a residual.  
- In the sum of residuals, we are adding up the **vertical deviations** (positive above the model, negative below). 

Students may also say things like “you’re summing the error” or “you’re adding up how far off the guesses are.” Push them to be spatial:  
- Are we adding lengths, deviations, or areas?

</div>


<div class="guided-notes">

### 4.6 Modify the R code below to calculate the sum of squares    

- NOTE: You need to square the residuals before you sum them
- Afterwards, write the R code to calculate the SS using the supernova() function
    
</div>


In [ ]:
# modify this code; you should get 1.66
resid(empty_model1)

# COMPLETE
sum(  resid(empty_model1)^2  )
# teach students the supernova function as well
supernova(empty_model1)

<div class="discussion-question">
    
### 4.7 Discussion Question: What units is this number in? Is 1.66 a lot or a little total error?
    
</div>

<div class="teacher-note">

**Sample Responses:**
- The units are squared seconds. 
- Some students may try to judge (e.g., 1.66 seems small). 

Key idea: SST is not easy to interpret on its own. The unit is not intuitive (what is a squared second?). SST only becomes meaningful when we use it to compare two different models of the same outcome variable.

The best answer to “Is 1.66 a lot?” is: “We don’t know until we compare it to another model.”
    
</div>

## 5 Mathematical Notation for SST

We don't introduce a lot of formulas in this course, but the formula for sum of squares is important enough that you should learn it. 

Here is the formula: <br><br>

<div style="font-size: 20pt;">$SST=\sum_{i=1}^n (Y_i-\bar{Y})^2$</div>

**This is often abbreviated like this:**<br><br>

<div style="font-size: 20pt;">$SST=\sum (Y_i-\bar{Y})^2$</div>

The $\sum$ symbol stands for sum (for every row i, which goes from 1 to n, the last row of your data).

**And you could also write this to mean the same thing:**<br><br>

<div style="font-size: 20pt;">$SST=\sum (Y_i-\hat{Y})^2$</div>

$\hat{Y}$ (y-hat) is the model prediction, which for the empty model is just the mean of $Y$, or $\bar{Y}$.

<div class="guided-notes">
    
### 5.1 Fill in the missing table cells to connect the R code to the formula
    
</div>

<table border="1" style="font-size: 18px; margin-left: 0; border-collapse: collapse; width: 100%">
  <thead>
    <tr>
      <th style="border: 1px solid black; font-weight: bold; width: 30%; text-align: left">R Code</th>
      <th style="border: 1px solid black; font-weight: bold; width: 10%; text-align: left">Formula Symbol</th>
      <th style="border: 1px solid black; font-weight: bold; width: 60%; text-align: left">Meaning</th>
    </tr>
  </thead>
  <tbody>
    <tr style="background-color: #f2f2f2;">
      <td style="border: 1px solid black;text-align: left; font-size: 16px;"><code>time_sec</code> or <code>student1$time_sec</code></td>
      <td style="border: 1px solid black;text-align: left">Y<sub>i</sub></td>
      <td style="border: 1px solid black;text-align: left">Values of the outcome variable (each guess in seconds)</td>
    </tr>
    <tr>
      <td style="border: 1px solid black;text-align: left; font-size: 16px;"><code>mean(student1&#36;time_sec)</code></td>
      <td style="border: 1px solid black;text-align: left;"><span style="text-decoration: overline;">Y</span><br>(Y-bar)</td>
      <td style="border: 1px solid black;text-align: left;"> </td>
    </tr>
    <tr style="background-color: #f2f2f2;">
      <td style="border: 1px solid black;text-align: left; font-size: 16px;"><code>_________(empty_model)</code></td>
      <td style="border: 1px solid black;text-align: left;">Y<sub>i</sub> − <span style="text-decoration: overline;">Y</span></td>
      <td style="border: 1px solid black;text-align: left;">Residuals from the empty model (differences between each value and the mean)</td>
    </tr>
    <tr>
      <td style="border: 1px solid black;text-align: left; font-size: 16px;"><code>resid(empty_model)^2</code></td>
      <td style="border: 1px solid black;text-align: left;"> </td>
      <td style="border: 1px solid black;text-align: left;">Squared residuals</td>
    </tr>
    <tr style="background-color: #f2f2f2;">
      <td style="border: 1px solid black;text-align: left; font-size: 16px;"><code>sum(resid(empty_model)^2)</code></td>
      <td style="border: 1px solid black;text-align: left;"> </td>
      <td style="border: 1px solid black;text-align: left;"> </td>
    </tr>
  </tbody>
</table>

<div class="discussion-question">

### 5.2 Discussion Question: What happens when we run the code in the cell below? Will it add additional rows or additional columns to the `student1` dataset? How do you know?

</div>

In [ ]:
# just run this; but let's not print student1 just yet
# we'll print it after our activity in 5.3
student1$empty_predict <- predict(empty_model1)
student1$empty_resid <- resid(empty_model1)
student1$empty_resid_sqrd <- resid(empty_model1)^2

<div class="teacher-note">

**Sample Responses:**  
- It adds columns, not rows.  
- You know this because the number of trials (rows) stays the same; we’re just attaching new information (predictions, residuals, squared residuals) for each trial.  

</div>

<div class="guided-notes">

### 5.3 Below is the new version of the student1 data table that we created using the predict() and resid() functions. Some of the cells are empty. 
    
- Fill in the top of each column with the R code that was used to produce each of the new columns (e.g., `student1$empty_predict <- <your code here>`)
- Fill in the second row of each column with the appropriate mathematical notation: 
 $Y_i$, $\hat{Y}$, $(Y_i-\hat{Y})$, or $(Y_i-\hat{Y})^2$.
- Finally, fill in any missing values in the data table.

</div>

<table border="1" style="font-size: 18px; margin-left: 0; border-collapse: collapse; width: 100%">
  <thead>
    <tr>
      <th colspan="2" style="border: 1px solid black; background-color: #7F7F7F; color: white; text-align: left; width: 20%;">
        R code &rarr;
      </th>
      <th style="border: 1px solid black; width: 26%;"></th>
      <th style="border: 1px solid black; width: 27%;"></th>
      <th style="border: 1px solid black; width: 27%;"></th>
    </tr>
      <tr>
      <th style="border: 1px solid black; background-color: #444; color: white; text-align: left; width: 20%;">
        math &rarr;
      </th>
      <th style="border: 1px solid black;text-align: right;"></th>
      <th style="border: 1px solid black; width: 26%;"></th>
      <th style="border: 1px solid black; width: 27%;"></th>
      <th style="border: 1px solid black; width: 27%;"></th>
    </tr>
    <tr>
      <th style="border: 1px solid black;text-align: right;">trial</th>
      <th style="border: 1px solid black;text-align: right;">time_sec</th>
      <th style="border: 1px solid black;text-align: right;">empty_predict</th>
      <th style="border: 1px solid black;text-align: right;">empty_resid</th>
      <th style="border: 1px solid black;text-align: right;">empty_resid_sqrd</th>
    </tr>
  </thead>
  <tbody>
    <tr style="border: 1px solid black;text-align: right;">
      <td style="border: 1px solid black;">1</td>
      <td style="border: 1px solid black;">7.4</td>
      <td style="border: 1px solid black;">6.8</td>
      <td style="border: 1px solid black;">0.6</td>
      <td style="border: 1px solid black;">0.36</td>
    </tr>
      <tr>
      <td style="border: 1px solid black;">2</td>
      <td style="border: 1px solid black;">6.0</td>
      <td style="border: 1px solid black;"> </td>
      <td style="border: 1px solid black;">-0.8</td>
      <td style="border: 1px solid black;">0.64</td>
    </tr>
    <tr>
      <td style="border: 1px solid black;">3</td>
      <td style="border: 1px solid black;">7.5</td>
      <td style="border: 1px solid black;">6.8</td>
      <td style="border: 1px solid black;"> </td>
      <td style="border: 1px solid black;">0.49</td>
    </tr>
    <tr>
      <td style="border: 1px solid black;">4</td>
      <td style="border: 1px solid black;">6.4</td>
      <td style="border: 1px solid black;">6.8</td>
      <td style="border: 1px solid black;">-0.4</td>
      <td style="border: 1px solid black;"> </td>
    </tr>
    <tr>
      <td style="border: 1px solid black;">5</td>
      <td style="border: 1px solid black;">6.7</td>
      <td style="border: 1px solid black;"> </td>
      <td style="border: 1px solid black;">0.1</td>
      <td style="border: 1px solid black;"> </td>
    </tr>
  </tbody>
</table>

In [ ]:
# just run this
student1$empty_predict <- predict(empty_model1)
student1$empty_resid <- resid(empty_model1)
student1$empty_resid_sqrd <- resid(empty_model1)^2

student1

<div class="discussion-question">
    
### 5.4 Now that we have this updated `student1` data frame, what are some different ways we could calculate Sum of Squares Total? 
    
- For each method, let's try to work out the R code to do implement.  
    
</div>

In [ ]:
# code here
sum(student1$empty_resid_sqrd)

<div class="teacher-note">

**Sample response:**  
- Students easily think that we should sum one of the columns. 
- But should we sum the `empty_resid_sqrd` or `empty_resid` columns? 

</div>

<div class="discussion-question">
    
### 5.5 The Sum of Squares Total for student 1's data is 1.66. Why does the first line of code work and the second doesn't?
    
</div>

In [ ]:
# this code works
sum( student1$empty_resid_sqrd ) 

# this code does not
# hint: this number is very close to 0
sum( student1$empty_resid )^2 

<div class="teacher-note">

**Teacher Note:**
It's a common error for students to add the residuals, then square the sum, instead of squaring each residual first and than adding the squares. If they do the first they will get 0 because the sum of the residuals is 0, and thus when you square it you still get 0!
    
The correct formula is this (as implemented by the first line of R code): $SST=\sum_{i=1}^n (Y_i-\bar{Y})^2$
    
The second line of R code is actually doing this: $SST = \left( \sum_{i=1}^n (Y_i - \bar{Y}) \right)^2$
    
</div>

## 6 How Sample Size Affects SST

We've been working with a small dataset from Student 1. It only has 5 data points. Student 2 generated 10 data points - twice as many as Student 1. Let's see how this difference in sample size affects Sum of Squares Total.

<div class="discussion-question">
    
### 6.1 Discussion: Student 2 happens to be a bit better at this task than Student 1. Student 1’s SST was 1.66. Let's make a prediction. How will Student 2's SST compare to Student 1?
    
</div>

<div class="teacher-note">

**Sample Responses:**  
- Some students will say "smaller SST" because Student 2 is better.  
- Some students will say "bigger SST" because there are 10 trials, so more squares get added up.  

**Follow-up questions (optional):**  
- If Students 1 and 2 were about equally good, how would their SSTs compare? (directs attention to "bigger SST" possibility)
- If Student 2 guessed exactly 7 seconds every time, what would their SST be? (directs attention to "bigger SST" possibility)

</div>


### 6.2 In the code cell below, fit and save `empty_model2` for `time_sec` in the `student2` data frame. Print it out and interpret the parameter estimate.

In [ ]:
# code here

# COMPLETE
empty_model2 <- lm(time_sec ~ NULL, data = student2)
empty_model2

<div class="teacher-note">

**Sample Response:**  
- 6.87 is the average time (in seconds) Student 2 guessed
- This number is a little closer to 7 seconds than Student 1's average (6.8)

</div>


<div class="discussion-question">
    
### 6.3 Discussion: Just looking at the two plots:
- Which one is Student 1 and which one is Student 2? 
- Which model, `empty_model1` and `empty_model2`, looks like it would have the larger SST? Why?
- Is Student 2 more accurate at the estimation task? How can you tell?

</div>

<img src="https://coursekata-course-assets.s3.us-west-1.amazonaws.com/UCLATALL/czi-stats-course/6.1-6.3-overview-SS-Total-2-students.jpg" alt = "the squares that comprise SST for student 1 and student 2's guesses" width="100%">

<div class="teacher-note">

**Sample Responses:**  
- Student 2’s SST looks bigger because there are way more squares being added up. 
- Student 1’s SST looks bigger because even though there are fewer squares, the squares are bigger.
- Key idea for them to notice is that SST depends both on size of square and number of squares.
- Some students may question whether Student 2 really is better.

</div>


### 6.4 Write R code to calculate Sum of Squares Total for `student1` and `student2`. Do the calculations confirm your intuitions?

In [ ]:
# code here

# COMPLETE
sum(resid(empty_model1)^2)
sum(resid(empty_model2)^2)

# alternatively you can use the shortcut
supernova(empty_model1)
supernova(empty_model2)

<div class="teacher-note">

**Teacher Note:**  
    
Student 2’s SST comes out larger than Student 1’s, even though Student 2’s individual squares are (on average) smaller. 

</div>


## 7 Variance

We just saw that Student 2’s SST was larger than Student 1’s. But this doesn’t mean the empty model fits Student 2’s data worse. It just means we added up more squared residuals (10 squares instead of 5).  

So SST by itself isn’t a fair way to compare across datasets of different sizes.

To get a more comparable indicator of how well the model fits, statisticians use something called **variance**. Variance is simply the SST divided by n-1 (the n stands for the number of data points). In this way, variance corrects for the difference in sample size between the two datasets. (n-1 is called the *degrees of freedom*. We will discuss this more in later parts of the course.)

<div class="guided-notes">
    
### 7.1 Modify the code we use to calculate SST around `empty_model2` to calculate variance instead for `student2`
    
- Afterwards, write the R code to calculate the SS using the supernova() function
- Finally, write the R code to calculate the variance using the var() function

</div>


In [ ]:
# modify this
sum(resid(empty_model2)^2)

# COMPLETE
sum(resid(empty_model2)^2) / (10-1) # or you can put 9

# the shortcut will print out the degrees of freedom and variance too 
# how super!
supernova(empty_model2)

# this code is like favstats
var(~ time_sec, data = student2)

<div class="discussion-question">

### 7.2 Discussion: In the `supernova()` table, the column labeled MS stands for "mean square".  That is just another way of saying "variance". Why does it make sense to think of variance as a "mean square"? 
</div>


<div class="teacher-note">

**Sample Responses:**  
- It's sort of like the area of the average square.
- It's similar to the average of the squared residuals (if we divided by n instead of n-1).

</div>


<div class="guided-notes">

### 7.3 How would you modify the formula for SST to get the formula for variance?   

<div style="font-size: 20pt;text-align: center">$\sum (Y_i-\bar{Y})^2$</div>

</div>


<div class="teacher-note">

**Sample Response:**  
    
<div style="font-size: 20pt;">$\text{variance} = \text{MS} = \frac{\sum (Y_i-\bar{Y})^2}{n-1}$</div>

</div>


### 7.4 Using variance to compare Student 1 and Student 2

In the code cell below, use the `supernova()` function to calculate the variance of Student 1 and Student 2.

In [ ]:
# code here
supernova(empty_model1)
supernova(empty_model2)

<div class="discussion-question">

### 7.5 Discussion: Notice that the SS of Student 2 is larger than Student 1; but the MS (variance) of Student 2 is smaller than Student 1. Why do you think this is? How would you interpret these numbers? 
</div>


<div class="teacher-note">

**Teacher Note:** SS is hard to interpret because it goes up with sample size. Variance, however, because it is adjusted by sample size, indicates that indeed, Student 2 is more accurate, on average, than Student 1, which means that the model has less error.

</div>


## 8 Practice What You've Learned
**NOTE: This might be better put in a separate notebook.**
Try collecting data on yourself, fitting the empty model, and then calculating the SST around your model. See how your level of error compares to your classmates, and think about what this comparison means.

### 8.1 Run the code in the cell below to set up your own stopwatch. How long is 16 seconds?

To make it harder, try estimating the duration of 16 seconds instead of 7. And try doing it 10 times. Just push **Start** to begin, and when you're done we'll show you how to download your data. **You won't be able to see your times until you have completed all 10 trials and imported the data into R.**

In [ ]:
# run this code to render the stopwatch
IRdisplay::display_html(sprintf(
  '<iframe src="https://uclatall.github.io/stopwatch/stopwatch-v3.html?condition=%s&class_id=%s&user_id=%s&trials=10&progress=hide&result=hide" width="500" height="220" sandbox="allow-scripts allow-same-origin"></iframe>',
  condition, class_id, user_id
))

### 8.2 Run the cell below to get your data in a data frame called `my_data`

Note: If you don't do the trials in 8.1 above, you will have no data in `my_data`.

In [ ]:
# run this code to import your data from your most recent session
sheet_url <- "https://docs.google.com/spreadsheets/d/e/2PACX-1vQRi4VBEFYGz_HnJXBsxlawbqM4vg5vgullb-iIqc5-dOpqEBCck12IS7lPD7bEdoxe8VPKYKe86Db4/pub?output=csv"
my_data <- utils::read.csv(sheet_url, header = TRUE, stringsAsFactors = FALSE) %>%
  mutate(date = as.POSIXct(date, format = "%Y-%m-%dT%H:%M:%OSZ", tz = "UTC")) %>%
  filter(user_id == !!user_id) %>%
  filter(session_id == session_id[which.max(date)]) %>%
  select(user_id, trial, time_sec)

# this prints out my_data
my_data

### 8.3 Create and save `my_empty_model` of `time_sec` in your data. Print out the parameter estimate.

In [ ]:
# code here
my_empty_model <- lm(time_sec ~ NULL, data = my_data)

### 8.4 Write your best-fitting empty model in GLM notation and interpret the parameter estimate

Empty model in GLM notation:

Interpretation of parameter estimate:


### 8.5 Plot `time_sec` by `trial`. Then:
- Overlay the empty model. Run it to make sure the code works.
- Overlay the residuals. Run it to make sure the code works.
- Overlay the squared residuals. Run it to make sure the code works.

In [ ]:
# code here

# COMPLETE
gf_point(time_sec ~ trial, data = my_data) %>%
  gf_model(my_empty_model) %>%
  gf_resid(my_empty_model) %>%
  gf_squaresid(my_empty_model)

### 8.6 Calculate SST and variance for `my_empty_model`

In [ ]:
# code here

# COMPLETE
sum(resid(my_empty_model)^2)
sum(resid(my_empty_model)^2) / 9

# or

supernova(my_empty_model)

<div class="discussion-question">
    
### 8.7 Discussion: Compare your performance with your classmates
- Who was most accurate on average?
- Who has the largest SST? Who has the smallest?
- Do people who are more accurate necessarily have smaller SSTs?
    
</div>